In [1]:
### Step 1: Initiation ###

from rdkit import Chem, rdBase
from rdkit.Chem import Draw,rdDepictor
from rdkit.Chem import AllChem, Descriptors, DataStructs
from rdkit.Chem.Draw import IPythonConsole, rdMolDraw2D
from rdkit.Chem.Scaffolds import rdScaffoldNetwork
from datetime import datetime
# import math
# import matplotlib as mpl
# from matplotlib import pyplot as plt
# import os
# import pyvis
# from pyvis.network import Network
# import inspect
import psycopg2
from collections import defaultdict
import sys
sys.path.append('/mnt/raid1/Python_projects')
sys.path.append('/mnt/raid1/Python_projects/pkg_mod')

# DATABASE Connections

# Chemistry Database
#db_chem = psycopg2.connect(host = "localhost", dbname="Chemistry", user="postgres", password="postgres")
db_chem = psycopg2.connect(host = "192.168.86.31", dbname="Chemistry", user="postgres", password="postgres")
# create a cursor
cur = db_chem.cursor()
# execute a statement
cur.execute('SELECT version()')
db_ver = cur.fetchone()
print('PostgreSQL database version:', db_ver) 
#display the PostgreSQL database server version


print('RDKit version: ',rdBase.rdkitVersion)
# print('MatplotLib version:', mpl.__version__)
# print('Pyvis version:', pyvis.__version__)
print(datetime.now())

###################################################################################################################

### Step 2: Generate Fingerprints for New Structures ("no fingerprints") ###

import collections
import fingerprinter as fpt

# SELECT Structures without fingerprints

# db_chem = psycopg2.connect(host = "localhost", dbname="Chemistry", user="postgres", password="postgres")
cur = db_chem.cursor() 
sql = 'SELECT s."Structure_ID", s."SMILES" FROM public."Structures" s \
WHERE s."Structure_ID" IN (SELECT f."Structure_ID" FROM public."Structures" f \
WHERE NOT EXISTS (SELECT 1 FROM public."Structure_Fingerprint" fp WHERE f."Structure_ID" = fp."Structure_ID"))' 
cur.execute(sql) 

mollist = cur.fetchall()
print()
print('##### Structures without Fingerprints #####')
print()
print(len(mollist),'new Structures')
print(mollist)


# generate Morgan fingerprints for molecules

fp_key = {}
fp_val = []
fp_dict = collections.defaultdict(list)
fp_allmol = []
fp_smi_list = {}

for mol in mollist:
    print()
    print('Molecule : ',mol)
    # print(mol[0], mol[1])
    m = Chem.MolFromSmiles(mol[1])

    # Generate Fingerprint SMILES and SMILES list
    fp_smi = fpt.FingerprintToSmiles(m, 3)
    print('Fingerprint : ',fp_smi)    
    for f in fp_smi: 
        fp_smi_list[f[0]] = f[1]
             
    # Generate Fingerprint Footprints:
    big ={}
    fp = AllChem.GetMorganFingerprint(m,3, bitInfo=big)
    # print(len(big))
    # print(big)
    # print(fp)
    for fpb,v in big.items():
        ma = []
        fp_dict[fpb].append(len(v))
        # print(mol[0],fpb, len(v),v)
        ma.append(mol[0])
        ma.append(fpb)
        ma.append(len(v))
        # print(ma)
        fp_allmol.append(ma)
        
print()
print("All Fingerprints by molecule")
print(fp_allmol)
        
print()        
print(fp_smi_list)
print(len(fp_smi_list),'SMILES')

for fpb,v in fp_smi_list.items():
    
    print(fpb,' - ',v)
    
        # key = fbp
        #fp_dict[fpb].append(len(v))
                
print()        
# print(fp_dict)
print(len(fp_dict),'fingerprints')

for fpb,v in fp_dict.items():
    print(fpb,' - ',len(v),' - ',v)
        # key = fbp
        #fp_dict[fpb].append(len(v))
        
###################################################################################################################

### Step 3: Insert new fingerprints into fingerprint table ###

print()
print('##### Insert New Fingerprints #####')
print()

# db_chem = psycopg2.connect(host = "localhost", dbname="Chemistry", user="postgres", password="postgres")
cur = db_chem.cursor()
sql = 'SELECT * FROM public."Fingerprints" WHERE "FP_Type" = 1'
cur.execute(sql)
fpdblist = cur.fetchall()
# print(len(fpdblist), "existing fingerprints")
# print(fpdblist)
# print(fp_smi_list)

print()
if len(fp_smi_list) == 0:
    print('No new fingerprint inserts')
else:
    print("New inserted fingerprints")
print()

sql = 'INSERT INTO public."Fingerprints"("FP_Smiles", "FP_Number", "FP_Type") VALUES(%s, %s, 1);'

for fpb,v in fp_smi_list.items():
    smi_fl = 0
    for i in range(0,len(fpdblist)):
        if fpb == fpdblist[i][3]:
            smi_fl = 1

    if smi_fl == 0:
        print(fpb,' - ',v, smi_fl, "inserted")
        cur.execute(sql, (v, fpb))
        db_chem.commit()
        #print(fpb,' - ',v, smi_fl, "inserted")

###################################################################################################################

### Step 4: Insert Fingerprints for New Molecules ###

print()
print('##### Insert Structure/Fingerprints Links #####')
print()

# db_chem = psycopg2.connect(host = "localhost", dbname="Chemistry", user="postgres", password="postgres")
cur = db_chem.cursor()
sql = 'SELECT * FROM public."Fingerprints" WHERE "FP_Type" = 1'
cur.execute(sql)
fpdblist = cur.fetchall()
# print(fpdblist)

fpid = {}

for m in fpdblist:
    # print(m)
    i = m[0]
    j = m[3]
    # print(i,j)
    fpid[j]=i
    
# print(fpid)

cur = db_chem.cursor()
sql = 'INSERT INTO public."Structure_Fingerprint"("Structure_ID","FP_ID","FP_Count") VALUES (%s,%s,%s);'

# Insert Notification
print()
if len(fp_allmol) == 0:
    print('No new structure/fingerprint inserts')
else:
    print('New structure/fingerprint links')
print()

for m in fp_allmol:
    # j = m[1]
    i = fpid[m[1]]
    cur.execute(sql, (m[0],i,m[2]))
    db_chem.commit()
    print(m,i)
    
###################################################################################################################

### Step 5: Update Shannon Entropies ###
print()
print('##### Shannon Update #####')
print()

# This is part of netprepper
#    import pandas as pd
#    from scipy import stats as scistat

# import sys
# print(sys.path)
# set PYTHONPATH to proper directory set holds private modules and packages (here pkg_mod)
import netprepper

# SELECT fingerprint length and append to fp_dict

#db_chem = psycopg2.connect(host = "localhost", dbname="Chemistry", user="postgres", password="postgres")
db_chem = psycopg2.connect(host = "192.168.86.31", dbname="Chemistry", user="postgres", password="postgres")
cur = db_chem.cursor() 
sql = 'SELECT sf."Structure_ID", sf."FP_ID", sf."FP_Count", fp."FP_Shannon" FROM public."Structure_Fingerprint" sf \
JOIN public."Fingerprints" fp ON sf."FP_ID" = fp."Fingerprint_ID" WHERE fp."FP_Type" = 1'
cur.execute(sql)
fp_list = cur.fetchall()
# print(fp_list)

fp_dict = collections.defaultdict(list)
fp_mol = collections.defaultdict(list)

for m in fp_list:
    # print(m, m[1],m[2])
    fp_dict[m[1]].append(m[2])
    fp_mol[m[0]].append(m[1])

# n_fp = 0
print(len(fp_dict),'fingerprints')
print(len(fp_mol),'structures')
print()

n_fp = len(fp_mol)

#for fpb, i in fp_dict.items():
#    n_fp = max(n_fp, len(i))
    # print (fpb, i, n)


# update entropy field in Fingerprints    
    
sql = 'UPDATE public."Fingerprints" SET "FP_Shannon" = %s WHERE "Fingerprint_ID" = %s'
cur = db_chem.cursor()
for fpb, i in fp_dict.items():
    ns = netprepper.shannon(i,n_fp)
    # print(fpb,' - ',netprepper.shannon(i,n))
    print(fpb,' - ',ns[0],' - ',ns[1])
    cur.execute(sql, (ns[1], fpb))
db_chem.commit()



PostgreSQL database version: ('PostgreSQL 12.8 (Ubuntu 12.8-1.pgdg20.04+1) on x86_64-pc-linux-gnu, compiled by gcc (Ubuntu 9.3.0-17ubuntu1~20.04) 9.3.0, 64-bit',)
RDKit version:  2020.03.3
2021-09-09 22:49:35.206422

##### Structures without Fingerprints #####

0 new Structures
[]

All Fingerprints by molecule
[]

{}
0 SMILES

0 fingerprints

##### Insert New Fingerprints #####


No new fingerprint inserts


##### Insert Structure/Fingerprints Links #####


No new structure/fingerprint inserts


##### Shannon Update #####

Net_Prepper_Imported
1036 fingerprints
69 structures

10  -  [1, 1, 1, 1, 1, 1, 1, 1, 3, 6, 6, 3, 3, 5, 5, 5, 5, 5, 1, 1, 1, 1, 2, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]  -  1.1010562539260735
7  -  [1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

44  -  [2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]  -  0.1512890335944373
45  -  [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]  -  0.07575108500341506
46  -  [2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]  -  0.07575108500341506
47  -  [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]  -  0.07575108500341506
48  -  [2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

96  -  [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]  -  0.07575108500341506
97  -  [1, 1, 1, 2, 3, 5, 1, 3, 4, 3, 4, 4, 1, 2, 1, 1, 2, 2, 2, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]  -  1.0486657596060356
98  -  [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]  -  0.07575108500341506
99  -  [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]  -  0.07575108500341506
100  -  [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

149  -  [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]  -  0.07575108500341506
150  -  [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]  -  0.07575108500341506
151  -  [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]  -  0.07575108500341506
152  -  [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]  -  0.07575108500341506
153  -  [1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0

201  -  [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]  -  0.07575108500341506
202  -  [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]  -  0.07575108500341506
203  -  [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]  -  0.07575108500341506
204  -  [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]  -  0.07575108500341506
205  -  [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

...

749  -  [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]  -  0.07575108500341506
750  -  [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]  -  0.07575108500341506
751  -  [2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]  -  0.07575108500341506
752  -  [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]  -  0.07575108500341506
753  -  [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

808  -  [2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]  -  0.07575108500341506
809  -  [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]  -  0.07575108500341506
810  -  [2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]  -  0.07575108500341506
811  -  [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]  -  0.07575108500341506
812  -  [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

870  -  [1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]  -  0.17884491271684755
871  -  [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]  -  0.07575108500341506
872  -  [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]  -  0.07575108500341506
873  -  [1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]  -  0.13119781096951133
874  -  [1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0

930  -  [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]  -  0.07575108500341506
931  -  [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]  -  0.07575108500341506
932  -  [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]  -  0.07575108500341506
933  -  [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]  -  0.07575108500341506
934  -  [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

989  -  [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]  -  0.07575108500341506
990  -  [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]  -  0.07575108500341506
991  -  [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]  -  0.07575108500341506
992  -  [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]  -  0.07575108500341506
993  -  [1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [33]:
db_chem = psycopg2.connect(host = "192.168.86.31", dbname="Chemistry", user="postgres", password="postgres")
cur = db_chem.cursor() 
sql = 'SELECT max("target") FROM public."Tanimoto" GROUP BY "target"'
cur.execute(sql)
t = cur.fetchone()
#print(tani_max, type(tani_max))
if t is None:
    tani_max = 0
else:
    tani_max = t[0]
#print(tani_max)

cur = db_chem.cursor() 
sql = 'SELECT "Structure_ID", "SMILES" FROM public."Structures" ORDER BY "Structure_ID"' 
cur.execute(sql) 
mollist = cur.fetchall()
print(mollist)

cur = db_chem.cursor() 
sql = 'SELECT max("Structure_ID") FROM public."Structures"' 
cur.execute(sql) 
m = cur.fetchone()
mol_max = m[0]
print(tani_max, mol_max)

if mol_max > tani_max:
    ml = len(mollist)
    tp = mollist[0][0]
    i = 0
    while tp < tani_max:
        tp = mollist[i][0]
        i = i+1
    print(ml,tp, i)
    tp = i
    for i in range(tp+1, ml):
        m2 = Chem.MolFromSmiles(mollist[i][1])
        fp2 = AllChem.GetMorganFingerprint(m2,2)
        #fp2 = Chem.RDKFingerprint(m2)
        for j in range (0, i):
            m1 = Chem.MolFromSmiles(mollist[j][1])
            fp1 = AllChem.GetMorganFingerprint(m1,2)
            #fp1 = Chem.RDKFingerprint(m1)
            tan = DataStructs.TanimotoSimilarity(fp1,fp2)
            if tan >= .5:
                print(i, j, tan, mollist[i][0], mollist[i][1], mollist[j][0], mollist[j][1])
            #print(fp1, fp2)
            
#fp1 = Chem.RDKFingerprint(ref)
#fp2 = Chem.RDKFingerprint(mol1)

[(10, 'CCO'), (11, 'CCN'), (13, 'CCC'), (14, 'CSC'), (15, 'CCSCC'), (16, 'CCCC'), (17, 'c1ccccc1'), (18, 'c1cnccc1'), (19, 'Cc1ccccc1'), (20, 'Cc1ccncc1'), (21, 'Cc1cccnc1'), (22, 'Cc1ccncc1Cl'), (23, 'Cc1c(N)cncc1Cl'), (24, 'CCOCC'), (34, 'CCNCC'), (35, 'CCSC'), (36, 'CCPCC'), (37, 'CCOP(=O)C'), (38, 'CCCNCC'), (39, 'O=C1NC(S/C1=C\\C2=CC=C(C3=C([N+]([O-])=O)C=CC=C3)O2)=N'), (40, 'OC1=CC=C(C(C)(C)C)C=C1/N=C/C2=CC=CC([N+]([O-])=O)=C2 '), (41, 'C1=CC=C(C(=C1)C2=NC(=NO2)C3=CC(=CC=C3)C(=O)O)F'), (42, 'C[C@H]1[C@H]2Cc3ccc(O)cc3[C@@]1(C)CCN2CCC#N'), (43, 'C[C@H]1[C@H]2Cc3ccc(O)cc3[C@@]1(C)CCN2CCCC#N'), (44, 'C#CCCN1CC[C@]2(C)c3cc(O)ccc3C[C@@H]1[C@@H]2C'), (45, 'C=CCCN1CC[C@]2(C)c3cc(O)ccc3C[C@@H]1[C@@H]2C'), (46, 'C=CCN1CC[C@]2(C)c3cc(O)ccc3C[C@@H]1[C@@H]2C'), (47, 'CCCCCCNc1nc(NC2CCN(C)CC2)c2cc(OC)c(OC)cc2n1'), (48, 'COc1cc2nc(N3CCOCC3)nc(NC3CCN(C)CC3)c2cc1OC'), (49, 'C1(=C2C(=NC=N1)[N](C=N2)[C@@H]3[C@@H]([C@H](CO3)O)O)O'), (50, 'C1=NC(=C2C(=N1)N(C=N2)[C@H]3[C@@H]([C@@H]([C@H](O3)COP(=O)(O)